In [47]:
%pylab inline
import numpy as np
import scipy as sc

#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("talk")
#rc('axes', labelsize=20, titlesize=20)

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import scipy.stats as ss

Populating the interactive namespace from numpy and matplotlib


In [48]:
def SimulateNormal(mean, sd, N):
    return np.random.normal(loc=mean, scale=sd, size=N)

def PlotPosteriorAgainstSample(priorMean, priorSd, likelihoodSd, sampledTheta, originalData):
    Hist = plt.hist(sampledTheta, sd(len(sampledTheta)), normed=True) #plot the hist for the sampled theta 
    mean = (priorMean/(priorSd^2) + np.sum(originalData)/likelihoodSd^2)/(1/priorSd^2 + len(originalData)/likelihoodSd^2)
    sd = (1/priorSd^2 + len(originalData)/likelihoodSd^2)^(-1)
    x = np.linspace(mean-3*sd, mean+3*sd, 100)
    trueDensity = plt.plot(x,mlab.normpdf(x, mean, sd)) #plot the true density with mean and sd               
    plt.show()


In [61]:
def ABC(priorSampler, likelihoodSimulator, summaryStatistics, epsilon, data, n): # epsilon is the number
    # data is a numpy.array (format), each element is one observation 
    # priorSampler and likelihoodSimulator return numpy.arrays, each element is a random variable
    # summaryStatistics returns a 1-dim array
    
    # priorSampler - a function taking one argument: n - the desired length of the sample, it returns an np.array
    # likelihoodSimulator - a function taking two argument: the desired number of observations and the current parameter, it returns an np.array 
    # summaryStatistics - a function taking one argument - data, returns a 1-dim array
    # epsilon - currently a number, not percentage,
    # data - an array
    # n - number of simulations (not the number of accepted samples)
    
    prior_sample = priorSampler(n)
    stat = summaryStatistics(data)
    theta_generated = []
    accepted = []
    output_list = []
    
    for i in range(n):
        # we assume data is a list of vectors and those vectors are columns of our 'data frame' with data
        print data.shape
        simulated_data = likelihoodSimulator(data.shape[1], prior_sample[i])
        print simulated_data
        break
        
        temporary_stat = summaryStatistics(simulated_data)
        # in the line below we are comparing sum of squares of the elements of temporary_stat - stat
        if np.sum(np.square(np.subtract(temporary_stat, stat)))<epsilon^2: # check here!
            accept = 1
        else: accept = 0
            
        output_dict = {'accept': accept, 'z':simulated_data, 'theta': prior_sample[i]} # added theta
        # seems more reasonable to add the theta at the end of function ...
        output_list.append(output_dict)
    
    df = pd.DataFrame(output_list)
    df['theta2'] = prior_sample
    return df

In [62]:
def NormalPriorSampler(n):
    return SimulateNormal(0,1,n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return SimulateNormal(param, 1,n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(0,1,10)
df = ABC(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, 0.1, data , 100)
print df

(10,)


IndexError: tuple index out of range

In [46]:
a =np.array([1,2,3])
b = np.array([1.1, 2.2, 3.3])
print a^2
print np.sum(np.square(a) )

[3 0 1]
14
